# MLV Segmenting and Clustering Neighborhoods in Toronto Capstone. 


#### Firstly we will install geopy, folium and bs4, in case that hasnt installed in our Jupyter Notebook.

In [ ]:
#
#!pip install geopy
#!pip install folium
#!pip install bs4

#### Now import all libraries that we will use on this excercise.

In [1]:
import requests # library to handle requests
import pandas as pd # library for data analsysis
import numpy as np # library to handle data in a vectorized manner
import random # library for random number generation
from geopy.geocoders import Nominatim # module to convert an address into latitude and longitude values

# libraries for displaying images
from IPython.display import Image 
from IPython.core.display import HTML 
    
# tranforming json file into a pandas dataframe library
from pandas.io.json import json_normalize
import folium # plotting library
from bs4 import BeautifulSoup #Library to scraping info. from a web portal such a wikipedia.
print('Libraries imported.')

Libraries imported.


#### We get only the table information from the wikipedia URL.

In [2]:
url=requests.get("https://en.wikipedia.org/wiki/List_of_postal_codes_of_Canada:_M").text
soup=BeautifulSoup(url,"html.parser")
tb = soup.find('table', class_='wikitable')


#### Now, we transform the table from wikipedia into a pandas dataframe.

In [3]:
rows = tb.find_all('tr')
l = []
for tr in rows:
    td = tr.find_all('td')
    row = [tr.text for tr in td]
    l.append(row)
df=pd.DataFrame(l, columns=["Postcode", "Borough","Neighbourhood"])
df=df.drop([0])
df=df.reset_index(drop=True)
#Replace dirty string:
df["Neighbourhood"]=df["Neighbourhood"].str.replace('\n', '')
df[0:5]

,Postcode,Borough,Neighbourhood
0,M1A,Not assigned,Not assigned
1,M2A,Not assigned,Not assigned
2,M3A,North York,Parkwoods
3,M4A,North York,Victoria Village
4,M5A,Downtown Toronto,Harbourfront


### Data Preparation Section:


#### Select all records without "Not assigned" in Borough and replace all the "Not assigned" Neighbourhood records with the correspondent Borough name.

In [4]:
df=df.loc[df["Borough"]!="Not assigned"]
index=df.loc[df["Neighbourhood"]=="Not assigned"].index.values.astype(int)
for i in index:
    df.at[i, 'Neighbourhood'] = df.at[i, 'Borough']

#### Set the dataframe to contain a record by Postcode. 
#### The Neighbourhood field must contain all neighbourhoods (coma separated) related to each PostCode

In [5]:

df=df.groupby(["Postcode", "Borough"])["Neighbourhood"].apply(lambda x:"%s" % ", ".join(x)).to_frame()
df.reset_index(level=1, inplace=True)
df.reset_index(level=0, inplace=True)
df[0:5]


,Postcode,Borough,Neighbourhood
0,M1B,Scarborough,"Rouge, Malvern"
1,M1C,Scarborough,"Highland Creek, Rouge Hill, Port Union"
2,M1E,Scarborough,"Guildwood, Morningside, West Hill"
3,M1G,Scarborough,Woburn
4,M1H,Scarborough,Cedarbrae


#### The shape of de dataframe 

In [6]:
df.shape

(103, 3)